# **Organización de Datos 75.06 - Trabajo Práctico N°1**
C3. Para variables continuas existe la entropía diferencial, un análogo al caso discreto. Para las productoras con más de 50 películas, ¿Cúal es la productora con más y menos entropía diferencial para su ROI (Return of Investment)? ¿Qué interpretación puede hacer de esto?

In [21]:
import pandas as pd
import numpy as np
from scipy.stats import differential_entropy

from google.colab import drive
drive.mount('/content/drive')

peliculas = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TP1/csv/movies.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:
df_limpio = peliculas[ ['title','revenue','budget','production_companies'] ].dropna()
df_limpio.production_companies = df_limpio.production_companies.map(lambda x: x.split(',')) #Separo las companias de c/ peli en una lista.
df_limpio = df_limpio.explode('production_companies')

df_limpio = df_limpio.loc[~((df_limpio['budget'].astype(int) == 0) | (df_limpio['revenue'].astype(int) == 0))].dropna() #Eliminamos ceros y NaN.
df_limpio['ROI'] = (df_limpio['revenue'] - df_limpio['budget'].astype(int)) / df_limpio['budget'].astype(int) #Calculamos ROI.

#df_limpio = df_limpio.groupby('production_companies').filter(lambda x : len(x)>50)
df_limpio = df_limpio[df_limpio.groupby('production_companies').production_companies.transform('count')>50].copy()

ROI_companias = df_limpio.groupby(['production_companies'], sort=False)['ROI'].apply(list).reset_index(name='ROI') #Agrupo todos los ROI de cada compania.
ROI_companias['entropia'] = ROI_companias.ROI.map(lambda x: differential_entropy(x)) #Calculo entropía.

max_compania = ROI_companias.loc[[ROI_companias['entropia'].idxmax()]].reset_index()
min_compania = ROI_companias.loc[[ROI_companias['entropia'].idxmin()]].reset_index()

display(max_compania)
display(min_compania)

,index,production_companies,ROI,entropia
0,5,60,"[5.072310931034483, 12.833333333333334, 4.9761...",4.663331


,index,production_companies,ROI,entropia
0,19,79,"[-0.6155358, 0.16715786666666665, -0.377554973...",1.770665
